# Data preparation

Features
- Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
- Vocal channel (01 = speech, 02 = song).
- Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
- Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
- Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
- Repetition (01 = 1st repetition, 02 = 2nd repetition).
- Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

NB We considered only the speech videos (vocal channel=01) with both audio and video (modality=01)

In [6]:
emotions = {1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}
emotional_intensity = {1:'normal', 2:'strong'}

sample_rate = 16000

In [1]:
import re
import os
import pandas as pd
import cv2
import numpy as np
from tensorflow import keras

In [2]:
path = "Datasets/RAVDESS/"

In [4]:
filenames = []
feats = []
labels = []
paths = []

for (dirpath, dirnames, fn) in os.walk(path):
    for name in fn:
        filename = name.split('.')[0]
        feat = filename.split('-')[2:]
        label = feat[0]
        filenames.append(filename)
        feats.append(feat)
        labels.append(label)
        paths.append(dirpath + '/' + filename)
        
filenames[:5]

['01-01-01-01-01-01-01',
 '01-01-01-01-01-02-01',
 '01-01-01-01-02-01-01',
 '01-01-01-01-02-02-01',
 '01-01-02-01-01-01-01']

## Data Exploration

In [7]:
df = pd.DataFrame(feats, columns = ['emotion', 'emotional intensity', 'statement', 'repetition', 'actor']).astype(int)

df['emotion'] = df['emotion'].map(emotions)
df['emotional intensity'] = df['emotional intensity'].map(emotional_intensity)

df['index'] = filenames
df.set_index('index', inplace=True)

df

,emotion,emotional intensity,statement,repetition,actor
index,,,,,
01-01-01-01-01-01-01,neutral,normal,1,1,1
01-01-01-01-01-02-01,neutral,normal,1,2,1
01-01-01-01-02-01-01,neutral,normal,2,1,1
01-01-01-01-02-02-01,neutral,normal,2,2,1
01-01-02-01-01-01-01,calm,normal,1,1,1
...,...,...,...,...,...
01-01-08-01-02-02-24,surprise,normal,2,2,24
01-01-08-02-01-01-24,surprise,strong,1,1,24
01-01-08-02-01-02-24,surprise,strong,1,2,24


## Export frames

- one frame every skip=3 starting from the 21th frame
- proportional resize to obtain height=224
- saved as png with and name videoname_iframe

In [34]:
def prepare_all_videos(filenames, paths, skip=1):
    nframes_tot = 0
    
    for count, video in enumerate(zip(filenames, paths)):
        # Gather all its frames
        save_frames(video[0], video[1], video[1].replace('RAVDESS', 'RAVDESS_frames'), skip)
        print(f"Processed videos {count+1}/{len(paths)}")
    return


def save_frames(filename, input_path, output_path, skip):
    # Initialize video reader
    cap = cv2.VideoCapture(input_path + '.mp4')
    frames = []
    count = 0
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    try:
        # Loop through all frames
        while True:
            # Capture frame
            ret, frame = cap.read()
            if (count % skip == 0 and count > 20):
                #print(frame.shape)
                if not ret:
                    break
                frame = cv2.resize(frame, (398, 224))
                cv2.imwrite(output_path + '/' + f'{filename}_{count}' + '.png', frame)
            count += 1
    finally:
        cap.release()
    return

In [36]:
prepare_all_videos(filenames, paths, skip=3)

Processed videos 1/1440
Processed videos 2/1440
Processed videos 3/1440
Processed videos 4/1440
Processed videos 5/1440
Processed videos 6/1440
Processed videos 7/1440
Processed videos 8/1440
Processed videos 9/1440
Processed videos 10/1440
Processed videos 11/1440
Processed videos 12/1440
Processed videos 13/1440
Processed videos 14/1440
Processed videos 15/1440
Processed videos 16/1440
Processed videos 17/1440
Processed videos 18/1440
Processed videos 19/1440
Processed videos 20/1440
Processed videos 21/1440
Processed videos 22/1440
Processed videos 23/1440
Processed videos 24/1440
Processed videos 25/1440
Processed videos 26/1440
Processed videos 27/1440
Processed videos 28/1440
Processed videos 29/1440
Processed videos 30/1440
Processed videos 31/1440
Processed videos 32/1440
Processed videos 33/1440
Processed videos 34/1440
Processed videos 35/1440
Processed videos 36/1440
Processed videos 37/1440
Processed videos 38/1440
Processed videos 39/1440
Processed videos 40/1440
Processed

Processed videos 321/1440
Processed videos 322/1440
Processed videos 323/1440
Processed videos 324/1440
Processed videos 325/1440
Processed videos 326/1440
Processed videos 327/1440
Processed videos 328/1440
Processed videos 329/1440
Processed videos 330/1440
Processed videos 331/1440
Processed videos 332/1440
Processed videos 333/1440
Processed videos 334/1440
Processed videos 335/1440
Processed videos 336/1440
Processed videos 337/1440
Processed videos 338/1440
Processed videos 339/1440
Processed videos 340/1440
Processed videos 341/1440
Processed videos 342/1440
Processed videos 343/1440
Processed videos 344/1440
Processed videos 345/1440
Processed videos 346/1440
Processed videos 347/1440
Processed videos 348/1440
Processed videos 349/1440
Processed videos 350/1440
Processed videos 351/1440
Processed videos 352/1440
Processed videos 353/1440
Processed videos 354/1440
Processed videos 355/1440
Processed videos 356/1440
Processed videos 357/1440
Processed videos 358/1440
Processed vi

Processed videos 637/1440
Processed videos 638/1440
Processed videos 639/1440
Processed videos 640/1440
Processed videos 641/1440
Processed videos 642/1440
Processed videos 643/1440
Processed videos 644/1440
Processed videos 645/1440
Processed videos 646/1440
Processed videos 647/1440
Processed videos 648/1440
Processed videos 649/1440
Processed videos 650/1440
Processed videos 651/1440
Processed videos 652/1440
Processed videos 653/1440
Processed videos 654/1440
Processed videos 655/1440
Processed videos 656/1440
Processed videos 657/1440
Processed videos 658/1440
Processed videos 659/1440
Processed videos 660/1440
Processed videos 661/1440
Processed videos 662/1440
Processed videos 663/1440
Processed videos 664/1440
Processed videos 665/1440
Processed videos 666/1440
Processed videos 667/1440
Processed videos 668/1440
Processed videos 669/1440
Processed videos 670/1440
Processed videos 671/1440
Processed videos 672/1440
Processed videos 673/1440
Processed videos 674/1440
Processed vi

Processed videos 953/1440
Processed videos 954/1440
Processed videos 955/1440
Processed videos 956/1440
Processed videos 957/1440
Processed videos 958/1440
Processed videos 959/1440
Processed videos 960/1440
Processed videos 961/1440
Processed videos 962/1440
Processed videos 963/1440
Processed videos 964/1440
Processed videos 965/1440
Processed videos 966/1440
Processed videos 967/1440
Processed videos 968/1440
Processed videos 969/1440
Processed videos 970/1440
Processed videos 971/1440
Processed videos 972/1440
Processed videos 973/1440
Processed videos 974/1440
Processed videos 975/1440
Processed videos 976/1440
Processed videos 977/1440
Processed videos 978/1440
Processed videos 979/1440
Processed videos 980/1440
Processed videos 981/1440
Processed videos 982/1440
Processed videos 983/1440
Processed videos 984/1440
Processed videos 985/1440
Processed videos 986/1440
Processed videos 987/1440
Processed videos 988/1440
Processed videos 989/1440
Processed videos 990/1440
Processed vi

Processed videos 1259/1440
Processed videos 1260/1440
Processed videos 1261/1440
Processed videos 1262/1440
Processed videos 1263/1440
Processed videos 1264/1440
Processed videos 1265/1440
Processed videos 1266/1440
Processed videos 1267/1440
Processed videos 1268/1440
Processed videos 1269/1440
Processed videos 1270/1440
Processed videos 1271/1440
Processed videos 1272/1440
Processed videos 1273/1440
Processed videos 1274/1440
Processed videos 1275/1440
Processed videos 1276/1440
Processed videos 1277/1440
Processed videos 1278/1440
Processed videos 1279/1440
Processed videos 1280/1440
Processed videos 1281/1440
Processed videos 1282/1440
Processed videos 1283/1440
Processed videos 1284/1440
Processed videos 1285/1440
Processed videos 1286/1440
Processed videos 1287/1440
Processed videos 1288/1440
Processed videos 1289/1440
Processed videos 1290/1440
Processed videos 1291/1440
Processed videos 1292/1440
Processed videos 1293/1440
Processed videos 1294/1440
Processed videos 1295/1440
P